# Converting ASDF Rotations to Rotation Matrices

To rotate objects in an ASDF scene,
you can use *azimuth*, *elevation* and *roll* angles,
for example like this:

```xml
<... rot="-30 12.5 5">
```

The used coordinate system conventions are shown in the
[section about position and orientation](position-orientation.rst).

In this section we show how these angles can be converted to
[rotation matrices](https://en.wikipedia.org/wiki/Rotation_matrix),
in order to practically use those rotations in software.

There isn't just a single way to choose rotation angles in 3D space,
in fact, there are very many ways to do this,
many of them leading to different rotation matrices.

Here's a (hopefully somewhat complete)
overview about the possible options and the choices taken by the ASDF:

* Right-handed vs. left-handed [coordinate system](https://en.wikipedia.org/wiki/Coordinate_system):
  The ASDF uses a right-handed one.

* Direction of the axes:
  The ASDF uses the "ENU" (east, north, up) convention.

* [Euler angles vs. Tait-Bryan angles](https://en.wikipedia.org/wiki/Euler_angles):
  The ASDF uses a variation of Tait-Bryan.

* There many [possible conventions](https://en.wikipedia.org/wiki/Axes_conventions)
  for the order of angles
  and which axes they rotate around:
  The ASDF conventions are shown in detail below.

* ["intrinsic"](https://en.wikipedia.org/wiki/Euler_angles#Definition_by_intrinsic_rotations)
  vs. ["extrinsic"](https://en.wikipedia.org/wiki/Euler_angles#Definition_by_extrinsic_rotations)
  = "local" vs. "global" reference system:
  This sounds complicated, but it's really just about the order of transformations.
  See below for details.

* [Rotating vectors (= "active" = "alibi")
  vs. rotating the coordinate system (= "passive" = "alias")](https://en.wikipedia.org/wiki/Active_and_passive_transformation):
  In the following derivations we consider the "active" situation,
  but a similar derivation can be done for the "passive" case.  
  In case you are wondering: the functions
  [sympy.matrices.rot_axis1()](https://docs.sympy.org/latest/modules/matrices/matrices.html#sympy.matrices.dense.rot_axis1)
  etc. do the latter, therefore we cannot use them here
  (at least not without some further manipulations).

* Rotation matrices can be derived for [pre-multiplication with column vectors
  vs. post-multiplication with row vectors](https://en.wikipedia.org/wiki/Rotation_matrix#Ambiguities):
  We are using column vectors here,
  but (different) matrices could be derived for use with row vectors.

Let's get started then, shall we?

First we import [SymPy](https://www.sympy.org/),
which is great for doing this kind of symbolic derivations:

In [ ]:
import sympy as sp

We have to define our three input angles.
These are often called
*azimuth/elevation/roll*,
or *yaw/pitch/roll*,
or *heading/elevation/bank*.

Here we just use the greek letters
$\alpha$, $\beta$ and $\gamma$:

In [ ]:
alpha, beta, gamma = sp.symbols('alpha beta gamma')

The ASDF uses an "ENU" (east, north, up) coordinate system
and the reference ("forward") direction is *north*,
i.e. along the positive y-axis.

In [ ]:
alpha

The *azimuth* angle $\alpha$ is:

* zero when pointing north (i.e. along the positive y-axis),
* rotating around the z-axis (which points up)
* positive when rotating towards west
  ([right hand rule](https://en.wikipedia.org/wiki/Right-hand_rule)).

In [ ]:
beta

The *elevation* angle $\beta$ is:

* zero in the horizontal plane,
* rotating around the *local* x-axis
* positive when the nose goes up (right hand rule).

In [ ]:
gamma

The *roll* angle $\gamma$ is:

* zero when the *top* of the object points to the zenith
  (which is just the normal "upright" orientation),
* rotating around the local y-axis
* positive when the object is leaning towards
  [starbord](https://en.wikipedia.org/wiki/Port_and_starboard)
  (right hand rule).

The definitions above use the "intrinsic" way of describing the rotations
(i.e. relative to "local" coordinate axes).

If you want to use the "extrinsic" way,
you can use the same angles.
You just have to choose the right order of ("global") rotations:
First *roll*, then *elevation*, then *azimuth*.
We will be using the "extrinsic" style below.

Let's also define the cartesian components of a vector $a$:

In [ ]:
a_x, a_y, a_z = sp.symbols('a_x:z')

We will need those only during the derivation,
they will not appear in the final equations.

## Azimuth: Rotation around the z-Axis

Writing the vector $a$ in cylindrical coordinates
$r_z$ (radius), $\phi_z$ (angle) and $a_z$ (height):

In [ ]:
r_z, phi_z = sp.symbols('r_z phi_z')

... we can get its cartesian coordinates like this:

In [ ]:
a = sp.Matrix([
    r_z * sp.cos(phi_z),
    r_z * sp.sin(phi_z),
    a_z,
])
a

We are using column vectors here,
that means we are searching for a rotation matrix
to left-multiply this vector in order to get the vector $b$.

To get a representation of the vector $b$,
let's rotate $a$ by an azimuth angle $\alpha$:

In [ ]:
b = sp.Matrix([
    r_z * sp.cos(phi_z + alpha),
    r_z * sp.sin(phi_z + alpha),
    a_z,
])
b

Note that $a_z$ is not affected by the rotation.

We can use some trigonometric identities
to expand this:

In [ ]:
b = b.expand(trig=True)
b

... and re-write it using the (cartesian) coordinates of vector $a$:
$a_x$, $a_y$ and $a_z$:

In [ ]:
b = b.subs(list(zip(a, [a_x, a_y, a_z])))
b

Remember, we are looking for a rotation matrix that,
when $a$ is left-multiplied by it, yields $b$.

In other words (or rather symbols):

\begin{equation}
\begin{bmatrix}
b_x\\
b_y\\
b_z
\end{bmatrix}
= R_z(\alpha)
\begin{bmatrix}
a_x\\
a_y\\
a_z
\end{bmatrix}
\end{equation}

Given the components of $b$ shown above,
we can simply pick out the matrix elements.

Or we let SymPy do it:

In [ ]:
Rz = sp.Matrix([[line.coeff(var) for var in [a_x, a_y, a_z]]
                for line in b])
Rz

That's it!

Let's do a little sanity check,
rotating the y unit vector (i.e. "looking straight ahead") by 90 degrees to the left:

In [ ]:
Rz.subs(alpha, sp.pi / 2) * sp.Matrix([0, 1, 0])

This yields the negative x unit vector, which points westwards.
That sounds about right!

## Elevation: Rotation around the (local) x-Axis

Now the same thing, just using a different vector $a$.

In [ ]:
r_x, phi_x = sp.symbols('r_x phi_x')
a = sp.Matrix([
    a_x,
    r_x * sp.cos(phi_x),
    r_x * sp.sin(phi_x),
])
a

Let's rotate $a$ by the elevation angle $\beta$
to get a vector $b$:

In [ ]:
b = sp.Matrix([
    a_x,
    r_x * sp.cos(phi_x + beta),
    r_x * sp.sin(phi_x + beta),
])
b

Again, expand using trig identities and substitute $a$ back in:

In [ ]:
b = b.expand(trig=True).subs(list(zip(a, [a_x, a_y, a_z])))
b

... and obtain a matrix $R_x(\beta)$ that transforms $a$ into $b$:

In [ ]:
Rx = sp.Matrix([[line.coeff(var) for var in [a_x, a_y, a_z]]
                for line in b])
Rx

And again a sanity check,
this time using an elevation of 90 degrees:

In [ ]:
Rx.subs(beta, sp.pi / 2) * sp.Matrix([0, 1, 0])

The result is a vector pointing up, which is what we expected,
didn't we?

## Roll: Rotation around the (local) y-Axis

Doing very similar steps as before:

In [ ]:
r_y, phi_y = sp.symbols('r_y phi_y')
a = sp.Matrix([
    r_y * sp.sin(phi_y),
    a_y,
    r_y * sp.cos(phi_y),
])
a

In [ ]:
b = sp.Matrix([
    r_y * sp.sin(phi_y + gamma),
    a_y,
    r_y * sp.cos(phi_y + gamma),
])
b

In [ ]:
b = b.expand(trig=True).subs(list(zip(a, [a_x, a_y, a_z])))
b

In [ ]:
Ry = sp.Matrix([[line.coeff(var) for var in [a_x, a_y, a_z]]
                for line in b])
Ry

Sanity check:
Applying a *roll* angle of 90 degrees
to a vector pointing up ...

In [ ]:
Ry.subs(gamma, sp.pi / 2) * sp.Matrix([0, 0, 1])

... leads to a vector pointing east. This is what we wanted.

## Combining all Axes

As mentioned above,
we have to choose the right sequence of (global) rotations:
first "roll", then "elevation", then "azimuth".

Note that we start with $R_y$ (roll) *on the right*,
and then left-apply $R_x$ (elevation)
and then left-apply $R_z$ (azimuth).

You should read this from right to left:

In [ ]:
R = Rz * Rx * Ry
R

That's it, that's our rotation matrix!

Copy this to use it with SymPy (you'll have to import `Matrix`, `sin` and `cos`
and define `alpha`, `beta` and `gamma`):

In [ ]:
print(R)

If you want to use it with NumPy, you can copy this
(you'll have to import `numpy` and define `alpha`, `beta` and `gamma`):

In [ ]:
from sympy.printing.pycode import NumPyPrinter
print(NumPyPrinter().doprint(R))

## Rotation Matrix to Angles

You may ask: how can we get back from the rotation matrix to our angles?

If you look at the matrix $R$ above,
you see that one component only depends on one variable.
Namely, the component in the last row, middle column:

In [ ]:
R[2, 1]

Therefore, we can get the value of $\beta$
simply by taking the arc-sine of this matrix element.
In a numeric calculation, this would probably look something like:

    beta = asin(R[2, 1])

The rest of the matrix components depend on more than one variable,
but there are a few elements that depend only on two variables.

If we divide the top middle component (multiplied by $-1$) by the one below:

In [ ]:
-R[0, 1] / R[1, 1]

... we get an expression that only depends on $\alpha$.

We can simplify this expression:

In [ ]:
_.simplify()

Therefore, to get the angle $\alpha$,
we only have to calculate
$\frac{-R_{0, 1}}{R_{1, 1}}$
and take the arc-tangent of the result.

To get the right quadrant of the result,
we will use the function [atan2()](https://en.wikipedia.org/wiki/Atan2)
in numeric calculations:

    alpha = atan2(-R[0, 1], R[1, 1])

We can do a similar thing to get $\gamma$:

In [ ]:
-R[2, 0] / R[2, 2]

In [ ]:
_.simplify()

Similar to above,
we take the arc-tangent of
$\frac{-R_{2, 0}}{R_{2, 2}}$
to get the angle $\gamma$.

    gamma = atan2(-R[2, 0], R[2, 2])

### Gimbal Lock

But wait a second, we might have a problem:
the dreaded [gimbal lock](https://en.wikipedia.org/wiki/Gimbal_lock)!

Let's consider the case where
$\beta = 90°$:

In [ ]:
R1 = R.subs(beta, sp.pi/2)
R1

If we try to calculate $\alpha$ and $\gamma$ like above,
we end up calculating

    atan2(0, 0)

Sadly, that is not defined:

In [ ]:
sp.atan2(0, 0)

We can try to find alternative equations
for $\alpha$ and $\gamma$
from the hitherto unused matrix elements
(but let's simplify the matrix first):

In [ ]:
R1 = sp.trigsimp(R1)
R1

In [ ]:
sp.simplify(R1[1, 0] / R1[0, 0])

In [ ]:
sp.simplify(R1[0, 2] / -R1[1, 2])

There is no unique solution to these equations.
You can freely choose either $\alpha$ or $\gamma$ and use that to calculate the other angle.

A very similar thing happens for
$\beta = -90°$:

In [ ]:
R2 = R.subs(beta, -sp.pi/2)
R2

In [ ]:
R2 = sp.trigsimp(R2)
R2

In [ ]:
sp.simplify(R2[1, 0] / R2[0, 0])

In [ ]:
sp.simplify(-R2[0, 2] / R2[1, 2])

Again, there is no unique solution.
You can freely choose one of the angles and then calculate the other one.

The easiest way to avoid this whole "gimbal lock" problem,
is simply to never convert rotation matrices to angles.